In [4]:
# imports and setup 
import pandas as pd
import scipy as sc
import numpy as np
from scipy.stats import norm

import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 6)
plt.style.use('ggplot')

In [5]:
# 'unzipped_data/air-quality-across-countries-in-covid19_aestheteaman01/USA.csv'
# 'unzipped_data/covid19-in-usa_sudalairajkumar/us_counties_covid19_daily.csv'
# 'unzipped_data/covid19-in-usa_sudalairajkumar/us_covid19_daily.csv'
# 'unzipped_data/covid19-in-usa_sudalairajkumar/us_states_covid19_daily.csv'

In [9]:
usa_air_q_file = 'unzipped_data/air-quality-across-countries-in-covid19_aestheteaman01/USA.csv'
df_aq_us = pd.read_csv(usa_air_q_file,
                       parse_dates=['Date'],
                       dayfirst=True,
                       index_col='Date')
df_aq_us.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 179365 entries, 2021-06-13 to 2021-08-15
Data columns (total 8 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   Country   179365 non-null  object 
 1   City      179365 non-null  object 
 2   Specie    179365 non-null  object 
 3   count     179365 non-null  int64  
 4   min       179365 non-null  float64
 5   max       179365 non-null  float64
 6   median    179365 non-null  float64
 7   variance  179365 non-null  float64
dtypes: float64(4), int64(1), object(3)
memory usage: 12.3+ MB


In [67]:
df_aq_us.sort_index()

,Country,City,Specie,count,min,max,median,variance
Date,,,,,,,,
2020-12-28,UNITED STATES,Denver,wind-speed,138,0.2,6.0,2.0,16.90
2020-12-28,UNITED STATES,Richmond,no2,96,1.4,12.4,5.0,50.28
2020-12-28,UNITED STATES,Providence,pressure,72,958.6,1027.3,1018.3,7984.64
2020-12-28,UNITED STATES,Queens,temperature,120,-3.0,50.1,7.8,2454.29
2020-12-28,UNITED STATES,Honolulu,co,21,5.6,7.8,5.6,4.71
...,...,...,...,...,...,...,...,...
2021-12-07,UNITED STATES,Salt Lake City,pm25,81,10.0,69.0,52.0,1742.50
2021-12-07,UNITED STATES,Little Rock,pm25,10,13.0,26.0,19.0,182.33
2021-12-07,UNITED STATES,Charlotte,wind-gust,34,1.5,14.5,10.8,102.15


In [64]:
df_la = df_aq_us[df_aq_us['City']=='Los Angeles'].sort_index().drop(['Country','City'], axis=1)
df_la

,Specie,count,min,max,median,variance
Date,,,,,,
2020-12-28,pm25,70,5.0,57.0,19.0,1761.05
2020-12-28,co,23,1.8,4.7,2.6,3.74
2020-12-28,temperature,93,5.8,13.0,8.8,29.11
2020-12-28,dew,20,6.0,9.0,8.0,7.50
2020-12-28,humidity,93,71.0,96.0,92.0,332.55
...,...,...,...,...,...,...
2021-12-07,dew,4,10.0,11.0,10.0,3.33
2021-12-07,pm10,7,22.0,34.0,30.0,205.71
2021-12-07,wind-gust,9,0.6,2.4,1.4,3.44


In [65]:
df_la['Specie'].value_counts()

pm25           323
co             323
no2            323
temperature    322
humidity       322
pressure       322
wind-gust      321
wind-speed     321
o3             320
pm10           319
dew            317
so2            223
Name: Specie, dtype: int64

In [38]:
df_la.pivot_table(columns=['Specie'])

Specie,co,dew,humidity,no2,o3,pm10,pm25,pressure,so2,temperature,wind-gust,wind-speed
count,23.801858,17.690852,75.074534,46.705882,22.756250,23.523511,68.671827,75.021739,5.067265,75.062112,35.090343,56.087227
max,7.819195,12.712934,89.530745,41.421981,42.292500,44.532915,95.024768,1017.277019,1.915247,24.613975,9.519003,6.377882
median,4.129102,10.385489,67.648447,18.688854,19.655000,27.705329,51.439628,1015.212112,1.727803,16.720186,2.175701,2.129595
min,2.341176,7.198107,38.357453,4.524768,5.636875,17.329154,15.380805,1010.306832,1.679372,12.509938,0.142679,0.116199
variance,36.246656,54.627729,2422.762671,1380.159071,1715.530062,734.952038,3677.995604,115.537236,1.122242,139.138230,96.015763,31.230779


In [71]:
df_ct = pd.read_csv('unzipped_data/covid19-in-usa_sudalairajkumar/us_counties_covid19_daily.csv',
                    parse_dates=True,
                    index_col='date')
df_ct

,county,state,fips,cases,deaths
date,,,,,
2020-01-21,Snohomish,Washington,53061.0,1,0.0
2020-01-22,Snohomish,Washington,53061.0,1,0.0
2020-01-23,Snohomish,Washington,53061.0,1,0.0
2020-01-24,Cook,Illinois,17031.0,1,0.0
2020-01-24,Snohomish,Washington,53061.0,1,0.0
...,...,...,...,...,...
2020-12-05,Sweetwater,Wyoming,56037.0,2098,10.0
2020-12-05,Teton,Wyoming,56039.0,1739,2.0
2020-12-05,Uinta,Wyoming,56041.0,1187,5.0


In [78]:
df_ct[(df_ct['state']=='California') & (df_ct['county']=='Los Angeles')]

,county,state,fips,cases,deaths
date,,,,,
2020-01-26,Los Angeles,California,6037.0,1,0.0
2020-01-27,Los Angeles,California,6037.0,1,0.0
2020-01-28,Los Angeles,California,6037.0,1,0.0
2020-01-29,Los Angeles,California,6037.0,1,0.0
2020-01-30,Los Angeles,California,6037.0,1,0.0
...,...,...,...,...,...
2020-12-01,Los Angeles,California,6037.0,408515,7700.0
2020-12-02,Los Angeles,California,6037.0,414304,7740.0
2020-12-03,Los Angeles,California,6037.0,422000,7782.0
